In [1]:
import numpy as np
import unyt as u
from rocketPy import Quaternion

In [2]:
class State(np.ndarray):

    def __new__(cls, input_array):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        # add the new attribute to the created instance
        
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # see InfoArray.__array_finalize__ for comments
        #if obj is None: return
        pass
    
    @classmethod
    def zero(cls):
        s = cls(np.zeros(14)) # create a state object
        s.quat = [0,1,0,0]
        return s
    
    def get_mass(self):
        return self[0]
    
    def set_mass(self, mass):
        self[0] = mass
        
    mass = property(get_mass, set_mass)
        
    def get_pos(self):
        return self[1:4]
    
    def set_pos(self, pos):
        self[1:4] = pos
        
    pos = property(get_pos, set_pos)
    
    def get_mom(self):
        return self[4:7]
    
    def set_mom(self, mom):
        self[4:7] = mom
        
    lin_mom = property(get_mom, set_mom)
    
    def get_quat(self):
        return Quaternion(*self[7:11])
    
    def set_quat(self, quat):
        if isinstance(quat, Quaternion):
            self[7:11] = quat.q
        else:
            self[7:11] = quat # as if it were a simple numpy array
        
    quat = property(get_quat, set_quat)
    
    def get_ang_mom(self):
        return self[11:14]
    
    def set_ang_mom(self, ang_mom):
        self[11:14] = ang_mom
        
    ang_mom = property(get_ang_mom, set_ang_mom)
    
    def __str__(self):
        if len(self)==14:
            s = f"State:\n mass={self[0]} \n position={self[1:4]} \n lin_mom={self[4:7]} \n quat={self[7:11]} \n ang_mom={self[11:14]} \n"
        else:
            s = self.__repr__()
        return s
    

In [3]:
q = Quaternion.from_angle(3*np.pi/180, [0,0,1])

In [4]:
q.q

array([0.99965732, 0.        , 0.        , 0.02617695])

In [5]:
s = State.zero()

In [6]:
s

State([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [7]:
s.mass=40
s

State([40.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.])

In [8]:
s.pos=[1,2,3]
s

State([40.,  1.,  2.,  3.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.])

In [9]:
s.lin_mom=[10,20,30]
s

State([40.,  1.,  2.,  3., 10., 20., 30.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.])

In [10]:
s.quat=q
s

State([4.00000000e+01, 1.00000000e+00, 2.00000000e+00, 3.00000000e+00,
       1.00000000e+01, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [11]:
s.ang_mom=[500,600,700]
s

State([4.00000000e+01, 1.00000000e+00, 2.00000000e+00, 3.00000000e+00,
       1.00000000e+01, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [12]:
s

State([4.00000000e+01, 1.00000000e+00, 2.00000000e+00, 3.00000000e+00,
       1.00000000e+01, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [13]:
print(s)

State:
 mass=40.0 
 position=State([1., 2., 3.]) 
 lin_mom=State([10., 20., 30.]) 
 quat=State([0.99965732, 0.        , 0.        , 0.02617695]) 
 ang_mom=State([500., 600., 700.]) 



In [14]:
s.mass

40.0

In [15]:
print(s)

State:
 mass=40.0 
 position=State([1., 2., 3.]) 
 lin_mom=State([10., 20., 30.]) 
 quat=State([0.99965732, 0.        , 0.        , 0.02617695]) 
 ang_mom=State([500., 600., 700.]) 



In [16]:
def mul2(t, state):
    return 2*state

In [17]:
import scipy.integrate as spint

In [36]:
sol=spint.solve_ivp(mul2, [0,3], s)
sol

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 38
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([0.        , 0.07701186, 0.57525844, 1.20968467, 1.89678211,
       2.60553874, 3.        ])
 t_events: None
        y: array([[4.00000000e+01, 4.66607427e+01, 1.26395424e+02, 4.49548555e+02,
        1.77641677e+03, 7.33009738e+03, 1.61339017e+04],
       [6.00000000e+00, 6.99911140e+00, 1.89593136e+01, 6.74322832e+01,
        2.66462515e+02, 1.09951461e+03, 2.42008525e+03],
       [7.00000000e+00, 8.16562996e+00, 2.21191992e+01, 7.86709971e+01,
        3.10872934e+02, 1.28276704e+03, 2.82343279e+03],
       [8.00000000e+00, 9.33214853e+00, 2.52790848e+01, 8.99097110e+01,
        3.55283354e+02, 1.46601948e+03, 3.22678034e+03],
       [5.00000000e+00, 5.83259283e+00, 1.57994280e+01, 5.61935694e+01,
        2.22052096e+02, 9.16262173e+02, 2.01673771e+03],
       [2.00000000e+01, 2.33303713e+01, 6.31977120e

In [43]:
tf = sol.t[-1]
tf

3.0

In [46]:
m0 = s.mass
m0

40.0

In [44]:
mf = sol.y[0][-1]
mf

16133.901685420298

In [21]:
ddt(s) = 2s
s = s0 exp(2 t)

State([4.00000000e+01, 1.00000000e+00, 2.00000000e+00, 3.00000000e+00,
       5.00000000e+00, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [57]:
np.testing.assert_approx_equal(m0*np.exp(2*tf),mf)

AssertionError: 
Items are not equal to 7 significant digits:
 ACTUAL: 16137.151739709405
 DESIRED: 16133.901685420298

In [23]:
pp

State([1., 2., 3.])

In [24]:
pp[2]=6

In [25]:
s

State([4.00000000e+01, 1.00000000e+00, 2.00000000e+00, 6.00000000e+00,
       5.00000000e+00, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [26]:
p = np.array([6, 7,8])*u.m

In [27]:
s.pos = p

In [28]:
s

State([4.00000000e+01, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
       5.00000000e+00, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [29]:
s.pos

State([6., 7., 8.])

In [30]:
p[1]=4

In [31]:
s

State([4.00000000e+01, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
       5.00000000e+00, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [32]:
s.quat

[0.99965732 0.         0.         0.02617695]

In [33]:
s.quat = q

In [34]:
s

State([4.00000000e+01, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
       5.00000000e+00, 2.00000000e+01, 3.00000000e+01, 9.99657325e-01,
       0.00000000e+00, 0.00000000e+00, 2.61769483e-02, 5.00000000e+02,
       6.00000000e+02, 7.00000000e+02])

In [35]:
s.quat

[0.99965732 0.         0.         0.02617695]